Load python packages

In [1]:
# I'm sure loads of these are unnecessary!

import pandas as pd
import numpy as np
import os
import glob
import copernicusmarine
from pathlib import Path
import xarray as xr
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from datetime import datetime
import netCDF4
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
from cartopy.util import add_cyclic_point
import matplotlib.colors as colors
import numpy as np

Load data from directories, establish some spatial variables

In [2]:
parent_dir    = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_dir      = os.path.join(parent_dir, 'data')
satellite_dir = os.path.join(data_dir, 'satellite')
output_dir = os.path.join(parent_dir, 'Output/sat_plot')
file_pattern  = os.path.join(satellite_dir, '*.nc')

satellite_files = glob.glob(file_pattern)

chla_data = xr.open_dataset(satellite_files[0]) # Unsure how this would behave with multiple netCDF files

#print(chla_data.variables.keys) #Uncomment to display netCDF file contents

min_lon = -35
max_lon = -5
min_lat = 55
max_lat = 66

In [3]:
display(chla_data)

<xarray.Dataset> Size: 3GB
Dimensions:              (time: 19, latitude: 960, longitude: 1440)
Coordinates:
  * latitude             (latitude) float32 4kB 55.01 55.02 ... 64.98 64.99
  * longitude            (longitude) float32 6kB -24.99 -24.98 ... -10.02 -10.01
  * time                 (time) datetime64[ns] 152B 2024-04-28 ... 2024-05-16
Data variables: (12/21)
    CHL                  (time, latitude, longitude) float32 105MB ...
    CHL_uncertainty      (time, latitude, longitude) float64 210MB ...
    DIATO                (time, latitude, longitude) float32 105MB ...
    DIATO_uncertainty    (time, latitude, longitude) float64 210MB ...
    DINO                 (time, latitude, longitude) float32 105MB ...
    DINO_uncertainty     (time, latitude, longitude) float64 210MB ...
    ...                   ...
    PICO_uncertainty     (time, latitude, longitude) float64 210MB ...
    PROCHLO              (time, latitude, longitude) float32 105MB ...
    PROCHLO_uncertainty  (time, latitude, longitude) float64 210MB ...
    PROKAR               (time, latitude, longitude) float32 105MB ...
    PROKAR_uncertainty   (time, latitude, longitude) float64 210MB ...
    flags                (time, latitude, longitude) int8 26MB ...
Attributes: (12/92)
    Conventions:                     CF-1.8, ACDD-1.3
    DPM_reference:                   GC-UD-ACRI-PUG
    IODD_reference:                  GC-UD-ACRI-PUG
    acknowledgement:                 The Licensees will ensure that original ...
    citation:                        The Licensees will ensure that original ...
    cmems_product_id:                OCEANCOLOUR_ATL_BGC_L3_NRT_009_111
    ...                              ...
    time_coverage_resolution:        P1D
    time_coverage_start:             2023-10-15T06:57:27Z
    title:                           cmems_obs-oc_atl_bgc-plankton_nrt_l3-mul...
    westernmost_longitude:           -46.0
    westernmost_valid_longitude:     -46.0
    copernicusmarine_version:        1.2.2

In [3]:


color = 'YlGnBu_r'

In [4]:
# define the caracteristics of the map and the features to plot
plotting = False
if plotting == True :
    for i in range(len(chla_data['time'].data)):
        plotname = output_dir + '/' + 'Chla' + str(i) + '.png'
        date_of_plot = chla_data['time'].data[i]
        plot_title = 'Chla ' + str(date_of_plot)
        f = plt.figure(figsize=(20, 10))
        ax = f.add_subplot(projection=ccrs.PlateCarree())

        ax.set_title(plot_title,fontsize=24)
        im = ax.pcolormesh(chla_data['longitude'].data, chla_data['latitude'].data,chla_data['CHL'].data[i,:],vmin = 0, vmax= 2,cmap = color,shading='nearest')
        ax.add_feature(cfeature.COASTLINE)
        sc = ax.scatter(position_df['LONGITUDE'], position_df['LATITUDE'], transform=ccrs.PlateCarree(), c = position_df['JULD'], zorder = 3)

        gl = ax.gridlines(draw_labels=True,x_inline=False,y_inline=False, crs=ccrs.PlateCarree())

        cbar = plt.colorbar(im, ax = ax, label=r'Chlorophyll a surface concentration (mg m$^{-3}$)')
        cbar.set_label('Chla', rotation=270, labelpad=15)
        plt.savefig(plotname)
        plt.close()

In [5]:
wmo_list = [4903532, 1902637]
#Float 1 = test float in the Icelandic Bassin
float_1_url = 'https://data-argo.ifremer.fr/dac/aoml/4903532/4903532_Sprof.nc'
#Float 2 = test float on Custard with glider next to it
float_2_url = 'https://data-argo.ifremer.fr/dac/coriolis/1902637/1902637_Sprof.nc'

#List the floats
floats_url = [float_1_url, float_2_url]

#Assign the local float directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
floats_dir =  os.path.join(parent_dir, 'Data/Floats')

#Create floats filename
floats_filenames = []
for i in floats_url:
    filename = floats_dir + '/' + i.rsplit('/', 1)[1]
    floats_filenames.append(filename)

In [6]:
position_df = pd.DataFrame({'PROF_NUM' : str(), 'LONGITUDE' : [], 'LATITUDE' : [], 'float' : int()})
for file, wmo in zip(floats_filenames, wmo_list):
    dat = xr.open_dataset(file)
    dat = dat.rename({'CYCLE_NUMBER':'PROF_NUM'}).swap_dims({'N_PROF':'PROF_NUM'})
    temp_df = dat[['LONGITUDE', 'LATITUDE', 'JULD']].to_dataframe().reset_index()
    temp_df['float'] = wmo
    position_df = pd.concat([position_df, temp_df], ignore_index=True)
    dat.close()

C:\Users\flapet\AppData\Local\Temp\ipykernel_3096\3027126715.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  position_df = pd.concat([position_df, temp_df], ignore_index=True)


In [7]:
import cartopy.feature as cfeature
import cartopy.crs as ccrs

land_50m = cfeature.NaturalEarthFeature('physical', 'land', '50m',
                                        edgecolor='k',
                                        facecolor=cfeature.COLORS['land'])

# Define data's extents I used an arbitrary extent that depicts the Icelandic Bassin
min_lon = -35
max_lon = -5
min_lat = 55
max_lat = 66


In [17]:
i = 10
plotname = output_dir + '/' + 'Chla' + str(i) + '.png'
date_of_plot = chla_data['time'].data[i]
plot_title = 'Chla ' + str(date_of_plot)
f = plt.figure(figsize=(20, 10))
ax = f.add_subplot(projection=ccrs.PlateCarree())

ax.set_title(plot_title,fontsize=24)
im = ax.pcolormesh(chla_data['longitude'].data, chla_data['latitude'].data,chla_data['CHL'].data[i,:],vmin = 0, vmax= 2,cmap = color)
ax.add_feature(cfeature.COASTLINE)
sc = ax.scatter(position_df['LONGITUDE'], position_df['LATITUDE'], transform=ccrs.PlateCarree(), c = 'black', zorder = 3)

for name, group in grouped:
    group.plot(x='LONGITUDE', y='LATITUDE', ax=ax, transform=ccrs.PlateCarree(), label=name, zorder=2)

gl = ax.gridlines(draw_labels=True,x_inline=False,y_inline=False, crs=ccrs.PlateCarree())

cbar = plt.colorbar(im, ax = ax, label=r'Chlorophyll a surface concentration (mg m$^{-3}$)')
cbar.set_label('Chla', rotation=270, labelpad=15)
plt.show()
plt.close()

In [9]:
grouped = position_df.groupby('float')

for i in range(len(chla_data['time'].data)):
    plotname = output_dir + '/' + 'Chla' + str(i) + '.png'
    date_of_plot = chla_data['time'].data[i]
    plot_title = 'Chla ' + str(date_of_plot)
    f = plt.figure(figsize=(20, 10))
    ax = f.add_subplot(projection=ccrs.PlateCarree())

    ax.set_title(plot_title,fontsize=24)
    im = ax.pcolormesh(chla_data['longitude'].data, chla_data['latitude'].data,chla_data['CHL'].data[16,:],vmin = 0, vmax= 2,cmap = color,shading='nearest')
    ax.add_feature(cfeature.COASTLINE)
    sc = ax.scatter(position_df['LONGITUDE'], position_df['LATITUDE'], transform=ccrs.PlateCarree(), c = 'black', zorder = 3)

    for name, group in grouped:
        group.plot(x='LONGITUDE', y='LATITUDE', ax=ax, transform=ccrs.PlateCarree(), label=name, zorder=2)

    gl = ax.gridlines(draw_labels=True,x_inline=False,y_inline=False, crs=ccrs.PlateCarree())

    cbar = plt.colorbar(im, ax = ax, label=r'Chlorophyll a surface concentration (mg m$^{-3}$)')
    cbar.set_label('Chla', rotation=270, labelpad=15)
    plt.savefig(plotname)
    plt.close()

KeyboardInterrupt: 